In [13]:
import pandas as pd
import numpy as np
import os

In [14]:
data_dirs = [i for i in os.listdir("data") if "scraping" not in i]

In [15]:
dfs = [pd.read_csv("data\\" + i) for i in data_dirs]

for idx, df in enumerate(dfs):
    df["version"] = idx

dfs = pd.concat(dfs)

In [20]:
dfs.groupby("version").agg(
    {"price":"mean", "price":"count"}
)

,price
version,
0,60
1,80
